In [19]:
"""
GTSAM Copyright 2010-2018, Georgia Tech Research Corporation,
Atlanta, Georgia 30332-0415
All Rights Reserved
Authors: Frank Dellaert, et al. (see THANKS for the full author list)
See LICENSE for the license information
An example of running visual SLAM using iSAM2.
Author: Duy-Nguyen Ta (C++), Frank Dellaert (Python)
"""
# pylint: disable=invalid-name, E1101

from __future__ import print_function

import gtsam
import gtsam.utils.plot as gtsam_plot
import matplotlib.pyplot as plt
import numpy as np
from gtsam.symbol_shorthand import L, X
from gtsam.examples import SFMdata
import time
import random#
from icecream import ic
%matplotlib widget
from ipywidgets import *
def visual_ISAM2_plot(result, gt_poses):
    """
    VisualISAMPlot plots current state of ISAM2 object
    Author: Ellon Paiva
    Based on MATLAB version by: Duy Nguyen Ta and Frank Dellaert
    """

    # Declare an id for the figure
    fignum = 0

    fig = plt.figure(fignum)
    axes = fig.gca(projection='3d')
    plt.cla()

    # Plot points
    # Can't use data because current frame might not see all points
    # marginals = Marginals(isam.getFactorsUnsafe(), isam.calculateEstimate())
    # gtsam.plot_3d_points(result, [], marginals)
    gtsam_plot.plot_3d_points(fignum, result, 'rx')

    # Plot cameras
    i = 0
    #2d result plo
    result_pose = []
    gt_pose = []
    
    while result.exists(i):
        pose_i = result.atPose3(i)
        gtsam_plot.plot_pose3(fignum, pose_i, 10)
        gtsam_plot.plot_pose3(fignum, gt_poses[i], 5)
        result_pose.append(np.array([pose_i.x(),pose_i.y(),pose_i.z()]))
        gt_pose.append(np.array([gt_poses[i].x(),gt_poses[i].y(),gt_poses[i].z()]))
        i += 1

    result_pose = np.array(result_pose)
    plt.plot(result_pose[:,0],result_pose[:,1],result_pose[:,2],'r')
    gt_pose = np.array(gt_pose)
    plt.plot(gt_pose[:,0],gt_pose[:,1],gt_pose[:,2],'b')
   

    # draw
    axes.set_xlim3d(-40, 40)
    axes.set_ylim3d(-40, 40)
    axes.set_zlim3d(-40, 40)
    axes.view_init(azim = -90,elev = 90)
    plt.pause(0.2)


In [20]:

plt.ion()

# Define the camera calibration parameters
K = gtsam.Cal3_S2(50.0, 50.0, 0.0, 50.0, 50.0)

# Define the camera observation noise model
measurement_noise = gtsam.noiseModel.Isotropic.Sigma(
    2, 5.0)  # one pixel in u and v

# Create the set of ground-truth landmarks
points = SFMdata.createPoints()
points += [p+3 for p in points]
# points += [p+3 for p in points]
points = [(i, point) for (i, point) in enumerate(points)]

ic(points)
# Create the set of ground-truth poses
poses = SFMdata.createPoses(K)
# Create an iSAM2 object. Unlike iSAM1, which performs periodic batch steps
# to maintain proper linearization and efficient variable ordering, iSAM2
# performs partial relinearization/reordering at each step. A parameter
# structure is available that allows the user to set various properties, such
# as the relinearization threshold and type of linear solver. For this
# example, we we set the relinearization threshold small so the iSAM2 result
# will approach the batch result.
parameters = gtsam.ISAM2Params()
parameters.setRelinearizeThreshold(0.01)
# parameters.relinearizeSkip = 1
isam = gtsam.ISAM2(parameters)

# Create a Factor Graph and Values to hold the new data
graph = gtsam.NonlinearFactorGraph()
initial_estimate = gtsam.Values()

# debug
optimize_time = 0
noise = 1


ic| points: [(0, array([10., 10., 10.])),
             (1, array([-10.,  10.,  10.])),
             (2, array([-10., -10.,  10.])),
             (3, array([ 10., -10.,  10.])),
             (4, array([ 10.,  10., -10.])),
             (5, array([-10.,  10., -10.])),
             (6, array([-10., -10., -10.])),
             (7, array([ 10., -10., -10.])),
             (8, array([13., 13., 13.])),
             (9, array([-7., 13., 13.])),
             (10, array([-7., -7., 13.])),
             (11, array([13., -7., 13.])),
             (12, array([13., 13., -7.])),
             (13, array([-7., 13., -7.])),
             (14, array([-7., -7., -7.])),
             (15, array([13., -7., -7.]))]


In [21]:
# find pose constraint bewteen pose 0 and pose 7
pose_0 = poses[0]
pose_7 = poses[7]
pose_m = pose_0.inverse()*pose_7
loop_closure = True


In [22]:

    #  Loop over the different poses, adding the observations to iSAM incrementally
    
    for i, pose in enumerate(poses):
        #if i==5:
        
        noise = 3
        #Add factors for each landmark observation
        for j, point in points:
            camera = gtsam.PinholeCameraCal3_S2(pose, K)
            measurement = camera.project(point) + np.random.normal(0, noise)
            
            graph.add(gtsam.GenericProjectionFactorCal3_S2(
                measurement, measurement_noise, i, 100 + j, K))
       
           
       
        # Add an initial guess for the current pose
        # Intentionally initialize the variables off from the ground truth
        if i<=3:
            # initial_estimate.insert(X(i), pose)
            initial_estimate.insert(i, pose.compose(gtsam.Pose3(
                gtsam.Rot3.Rodrigues(-0.1, 0.2, 0.15), gtsam.Point3(1, -1, 1))))
        else:
            initial_estimate.insert(i, current_estimate.atPose3(i-1))
        if i == 7 and loop_closure == True:
            print("loop closure!")
            covariance = np.eye(6)
            odometry_noise = gtsam.noiseModel.Gaussian.Covariance(covariance)
            R = np.eye(3)
            rotation = gtsam.Rot3(R)
            P3 = gtsam.Pose3(rotation,np.array([-0.1,-0.1,-0.1]).reshape(-1,1))
            graph.add(gtsam.BetweenFactorPose3(0,7,pose_m,odometry_noise))
        # # If this is the first iteration, add a prior on the first pose to set the
        # coordinate frame and a prior on the first landmark to set the scale.
        # Also, as iSAM solves incrementally, we must wait until each is observed
        # at least twice before adding it to iSAM.
        if i == 0:
            # Add a prior on pose x0
            pose_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array(
                [0.1, 0.1, 0.1, 0.3, 0.3, 0.3]))  # 30cm std on x,y,z 0.1 rad on roll,pitch,yaw
            graph.push_back(gtsam.PriorFactorPose3(0, poses[0], pose_noise))

            # Add a prior on landmark l0
            point_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.1)
            graph.push_back(gtsam.PriorFactorPoint3(
                100, points[0][1], point_noise))  # add directly to graph

            # Add initial guesses to all observed landmarks
            # Intentionally initialize the variables off from the ground truth
            for j, point in points:
                initial_estimate.insert(100+j, gtsam.Point3(
                    point[0]-0.25, point[1]+0.20, point[2]+0.15))
         
        else:
            # Update iSAM with the new factors
            start = time.time()
            isam.update(graph, initial_estimate)
            # Each call to iSAM2 update(*) performs one iteration of the iterative nonlinear solver.
            # If accuracy is desired at the expense of time, update(*) can be called additional
            # times to perform multiple optimizer iterations every step.
            isam.update()
            current_estimate = isam.calculateEstimate()
            optimize_time += time.time()-start
            print("****************************************************")
            print("Frame", i, ":")
            # for j in range(i + 1):
            #     print(X(j), ":", current_estimate.atPose3(X(j)))

            # for j in range(len(points)):
            #     print(L(j), ":", current_estimate.atPoint3(L(j)))

            # print('===================')
            # print('estimated: ', current_estimate.atPose3(X(i)))

            # print('gt: ', pose)

            visual_ISAM2_plot(current_estimate, poses)

            # Clear the factor graph and values for the next iteration
            graph.resize(0)
            initial_estimate.clear()

    print('optimize_time: ', optimize_time)
    # plt.ioff()
    # plt.show()


loop closure!
loop closure!


RuntimeError: Attempting to at the key "7", which does not exist in the Values.

In [ ]:
# #2d result plo
# result = current_estimate
# i = 0
# result_pose = []
# while result.exists(i):
#     pose_i = result.atPose3(i)
#     result_pose.append(np.array([pose_i.x(),pose_i.y()]))
#     i=i+1
# result_pose = np.array(result_pose)
# print(result_pose[:,0]) if i == 7:
            # print("loop closure!")
            # covariance = np.eye(6)
            # odometry_noise = gtsam.noiseModel.Gaussian.Covariance(covariance)
            # R = np.eye(3)
            # rotation = gtsam.Rot3(R)
            # P3 = gtsam.Pose3(rotation,np.array([-0.1,-0.1,-0.1]).reshape(-1,1))
            # graph.add(gtsam.BetweenFactorPose3(0,7,pose_m,odometry_noise))
# plt.show()